In [14]:
# warnings....ignore
import warnings

warnings.simplefilter(action='ignore', category=FutureWarning)

import pandas as pd

pd.options.mode.chained_assignment = None

In [15]:
DATA = []
BUCKETS = [0.005, 0.01, 0.025, 0.05, 0.1, 0.25, 0.5, 1, 2]
TIME_VECTOR = 5  # second

In [16]:
import json


def get_data_from_file(filename: str):
    f = open(filename, "r")
    while True:
        line = f.readline()
        if not line:
            break
        # Standardize json to dict
        row = json.loads(line)
        # Append to Array
        DATA.append(row)

In [17]:
get_data_from_file("../statics/request_data_with_anomaly_pod_instance.json")

In [18]:
group_by_el = ["path", "method", "service_name", "controller_pod"]

In [19]:
# Create dataframe
root_df = pd.DataFrame(DATA)

In [20]:
root_df.head()

,timestamp,path,method,service_name,controller_pod,duration_time
0,"01/02/2024, 10:42:25",/users,GET,process_log,pod-2,0.395
1,"01/02/2024, 10:42:26",/users,GET,process_log,pod-2,0.669
2,"01/02/2024, 10:42:26",/home,GET,process_log,pod-2,0.511
3,"01/02/2024, 10:42:26",/users,GET,process_log,pod-2,0.439
4,"01/02/2024, 10:42:27",/home,GET,process_log,pod-1,0.236


In [25]:

arr_data = []


# BUCKETS = [0.005, 0.01, 0.025, 0.05, 0.1, 0.25, 0.5, 1, 2]
def bucketing(row_data: pd.Series):
    duration_time = row_data["duration_time"]
    for bucket in BUCKETS:
        if duration_time <= bucket:
            row = {
                "timestamp": row_data.timestamp,
                "path": row_data.path,
                "method": row_data.method,
                "service_name": row_data.service_name,
                "controller_pod": row_data.controller_pod,
                "duration_time": duration_time,
                "le": str(bucket)
            }
            arr_data.append(row)
    # Always append bucket +Inf
    arr_data.append({
        "timestamp": row_data.timestamp,
        "path": row_data.path,
        "method": row_data.method,
        "service_name": row_data.service_name,
        "duration_time": duration_time,
        "controller_pod": row_data.controller_pod,
        "le": "+Inf"
    })
    # print(arr_data)


root_df.apply(lambda x: bucketing(x), axis=1)
df_with_bucketed = pd.DataFrame.from_records(arr_data)
df_with_bucketed

,timestamp,path,method,service_name,controller_pod,duration_time,le
0,"01/02/2024, 10:42:25",/users,GET,process_log,pod-2,0.395,0.5
1,"01/02/2024, 10:42:25",/users,GET,process_log,pod-2,0.395,1
2,"01/02/2024, 10:42:25",/users,GET,process_log,pod-2,0.395,2
3,"01/02/2024, 10:42:25",/users,GET,process_log,pod-2,0.395,+Inf
4,"01/02/2024, 10:42:26",/users,GET,process_log,pod-2,0.669,1
...,...,...,...,...,...,...,...
143730,"01/02/2024, 14:38:01",/users,GET,process_log,pod-2,0.363,+Inf
143731,"01/02/2024, 14:38:02",/users,GET,process_log,pod-2,0.274,0.5
143732,"01/02/2024, 14:38:02",/users,GET,process_log,pod-2,0.274,1
143733,"01/02/2024, 14:38:02",/users,GET,process_log,pod-2,0.274,2


In [22]:
from datetime import datetime

df_with_bucketed['timestamp'] = df_with_bucketed['timestamp'].apply(
    lambda x: datetime.strptime(x, "%d/%m/%Y, %H:%M:%S"))
# Convert to datetime pandas and set datetime columns is index for dataframe
df_with_bucketed['timestamp'] = pd.to_datetime(root_df['timestamp'])
df_with_bucketed = df_with_bucketed.set_index('timestamp')

In [23]:
df_with_bucketed.query("path == '/home' and controller_pod == 'pod-1' and le == '+Inf'")

,path,method,service_name,controller_pod,duration_time,le
timestamp,,,,,,
2024-01-02 10:42:32,/home,GET,process_log,pod-1,0.236,+Inf
2024-01-02 10:42:47,/home,GET,process_log,pod-1,0.462,+Inf
2024-01-02 10:42:57,/home,GET,process_log,pod-1,0.179,+Inf
2024-01-02 10:42:58,/home,GET,process_log,pod-1,0.585,+Inf
2024-01-02 10:43:07,/home,GET,process_log,pod-1,0.753,+Inf
...,...,...,...,...,...,...
NaT,/home,GET,process_log,pod-1,0.648,+Inf
NaT,/home,GET,process_log,pod-1,0.387,+Inf
NaT,/home,GET,process_log,pod-1,0.608,+Inf


In [24]:
df_with_bucketed['count'] = df_with_bucketed.groupby(
    ['path', 'method', 'controller_pod', 'service_name', 'le']).cumcount() + 1
df_with_bucketed.groupby([pd.Grouper(freq="30s"), 'path', 'method', 'controller_pod', 'service_name', 'le']).agg(
    {'count': 'last'}).reset_index()[['timestamp', 'path', 'method', 'controller_pod', 'service_name', 'le', 'count']].to_csv('request_data_with_anomaly_pod_instance.csv')